# Day 2 – Web Summarizer Using Ollama (Local Open-Source Model)

This notebook is a solution to the Day 2 homework assignment:

> Upgrade the Day 1 project to summarize a webpage to use an open-source model running locally via **Ollama** rather than **OpenAI's cloud APIs**.

We reuse the same structure from Day 1:

1. A function to fetch website contents
2. System and user prompts
3. A `messages_for(...)` helper
4. A `summarize(url)` function
5. A `display_summary(url)` helper that renders markdown


## 1. Imports and Ollama client setup

We will call Ollama through its **OpenAI-compatible** HTTP endpoint:

- Base URL: `http://localhost:11434/v1/`
- Model: `llama3.2` (or `llama3.2:1b` for smaller machines)

Note: Ollama does not require a real API key, but the `openai` client expects something, so we pass a dummy string like `"ollama"`.


In [ ]:
import os
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

# Ollama OpenAI-compatible endpoint
OLLAMA_BASE_URL = "http://localhost:11434/v1/"

# Choose the Ollama model you pulled, e.g.:
#   ollama pull llama3.2
# or for smaller machines:
#   ollama pull llama3.2:1b
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "llama3.2")

# Dummy key is fine – Ollama ignores it but the client requires one
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")


## 2. Prompts and message builder (same concept as Day 1)

We keep the same structure as Day 1, just changing the tone slightly if desired.


In [ ]:
system_prompt = """
You are a smart assistant that analyzes the contents of a website
and provides a short, clear, slightly humorous summary, ignoring text that is purely navigation.
Respond in markdown. Do not wrap the markdown in a code block – respond just with the markdown.
"""

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

def messages_for(website: str):
    """Build messages in the same format as Day 1, but for Ollama."""
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website},
    ]


## 3. Summarization and display helpers

These follow the same pattern as Day 1, but call `ollama.chat.completions.create` instead of the OpenAI cloud endpoint.


In [ ]:
def summarize(url: str) -> str:
    """Fetch the website contents and ask the local Ollama model to summarize it."""
    website = fetch_website_contents(url)
    response = ollama.chat.completions.create(
        model=OLLAMA_MODEL,
        messages=messages_for(website),
    )
    return response.choices[0].message.content

def display_summary(url: str):
    """Convenience helper to show the summary nicely in the notebook output."""
    summary = summarize(url)
    display(Markdown(summary))


## 4. Example usage

Once you have Ollama running and have pulled the model (for example `llama3.2`), you can test the summarizer below.


In [ ]:
# Try summarizing a site of your choice
# Example:
# display_summary("https://edwarddonner.com")

display_summary("https://edwarddonner.com")
